<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# YouTube - Send video stats to Notion
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/YouTube/YouTube_Get_statistics_from_video.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #youtube #video #statistics #naas_drivers #content #snippet #dataframe

**Author:** [Maxime Jublou](https://www.linkedin.com/in/maximejublou)

This notebook will synchronize your YouTube videos stats with a Notion database.

## Input

### Import library

In [1]:
import naas
from naas_drivers import youtube, notion
import pandas as pd
import pydash
import re
import regex
try:
    import emoji
except:
    ! pip install --user emoji
    import emoji

### Variables

To know how to generate a YouTube api key you can [watch this video](https://www.youtube.com/watch?v=ltdJOX_DVtE).

<a href='https://docs.naas.ai/drivers/notion'>How to get your Notion integration token ?</a>

In [2]:
# Youtube api Key
YOUTUBE_API_KEY = naas.secret.get('YOUTUBE_API_KEY') or 'YourYoutubeApiKey'

# Channel ID
channel_url = "https://www.youtube.com/channel/UCKKG5hzjXXU_rRdHHWQ8JHQ"

# Notion token
NOTION_TOKEN = naas.secret.get('NOTION_TOKEN') or 'YourNotionToken'

# Notion database url
notion_database_url = "https://www.notion.so/naas-official/ed622cae89e045249c464a08dc818876?v=989e444993d3421c8712e6e6b2d60810"

### Scheduling

Lets run this notebook every hour.

In [3]:
#naas.scheduler.add(cron="0 * * * *")

## Model

### Connect YouTube driver

In [4]:
youtube.connect(YOUTUBE_API_KEY)

### Get all uploads

In [5]:
df_uploads = youtube.channel.get_uploads(channel_url=channel_url)
df_uploads

,CHANNEL_ID,PLAYLIST_ID,VIDEO_ID,VIDEO_TITLE,VIDEO_DESCRIPTION,VIDEO_PUBLISHEDAT,VIDEO_THUMBNAILS,PUBLISHEDAT
0,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,ltdJOX_DVtE,Create a YouTube API Key (In less than 2 minutes),This short video will show you how to create a...,2022-06-03T07:52:29Z,https://i.ytimg.com/vi/ltdJOX_DVtE/maxresdefau...,2022-06-03 07:52:29+00:00
1,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,TIQiMTpzH-Q,⚡️🌎 Naas Community Call (2022/03/03),Here is what we are going to talk about:\n\n⚡️...,2022-03-07T21:00:00Z,https://i.ytimg.com/vi/TIQiMTpzH-Q/maxresdefau...,2022-03-07 21:00:00+00:00
2,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,1kjxWtgeV3E,💬 Using NLP to extract and summarize transcrip...,🌍💬 Interested in Open Source + NLP but don't k...,2021-10-15T18:17:04Z,https://i.ytimg.com/vi/1kjxWtgeV3E/maxresdefau...,2021-10-15 18:17:04+00:00
3,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,lvZKZx9ngrc,Code is power - A brief history of coding.,#code #opensource #power #office #hours #naas,2021-10-03T18:07:15Z,https://i.ytimg.com/vi/lvZKZx9ngrc/maxresdefau...,2021-10-03 18:07:15+00:00
4,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,4ds2FDI_60g,Push Jupyter notebooks to Production ⚡️ with 5...,#naas #notebooks #lowcode #opensource\n\n✊⚡️Pu...,2021-09-16T16:33:00Z,https://i.ytimg.com/vi/4ds2FDI_60g/maxresdefau...,2021-09-16 16:33:00+00:00
5,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,tQd-4AsHvqU,Is an idea environmentally friendly? 🌎👍 👎 (Hug...,#huggingface #transformers #ai #model #jupyter...,2021-09-06T17:00:44Z,,2021-09-06 17:00:44+00:00
6,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,jmC5VDN-MOI,🚨 Open Source Contribution step by step (Githu...,#opensource #contribution #git #naas #notebook...,2021-09-01T19:01:49Z,https://i.ytimg.com/vi/jmC5VDN-MOI/maxresdefau...,2021-09-01 19:01:49+00:00
7,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,1CfW0N8x8og,🧨 👿 What is wrong with Data Project? ...and wh...,#data #dataops #devops #agile #businesteam #te...,2021-08-17T15:09:55Z,https://i.ytimg.com/vi/1CfW0N8x8og/maxresdefau...,2021-08-17 15:09:55+00:00
8,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,W02XVb8IbGA,⏱1-minute pitch video for Pioneer,A 1-minute video about Naas project.,2021-08-09T16:41:49Z,https://i.ytimg.com/vi/W02XVb8IbGA/maxresdefau...,2021-08-09 16:41:49+00:00
9,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,NjUhb92i9eM,🌎🔗Making sense of LinkedIn data: how Mark used...,#socialmedia #linkedin #datamining #datascienc...,2021-07-27T18:07:10Z,https://i.ytimg.com/vi/NjUhb92i9eM/maxresdefau...,2021-07-27 18:07:10+00:00


### Enrich DataFrame with each video statistics

In [6]:
all_video_stats = None

for _, upload in df_uploads.iterrows():
    video_id = upload['VIDEO_ID']
    df_video_stats = youtube.video.get_statistics(f'https://www.youtube.com/watch?v={video_id}')
    if all_video_stats is None:
        all_video_stats = df_video_stats
    else:
        all_video_stats = pd.concat([all_video_stats, df_video_stats])
        
all_video_stats['VIDEO_ID'] = all_video_stats['ID']
df_uploads = df_uploads.merge(all_video_stats, on='VIDEO_ID')
df_uploads

,CHANNEL_ID_x,PLAYLIST_ID,VIDEO_ID,VIDEO_TITLE,VIDEO_DESCRIPTION,VIDEO_PUBLISHEDAT,VIDEO_THUMBNAILS,PUBLISHEDAT_x,CHANNEL_ID_y,CHANNEL_TITLE,...,DESCRIPTION,VIEWCOUNT,LIKECOUNT,DISLIKECOUNT,FAVORITECOUNT,COMMENTCOUNT,DURATION,DURATION_SECONDS,THUMBNAILS,PUBLISHEDAT_y
0,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,ltdJOX_DVtE,Create a YouTube API Key (In less than 2 minutes),This short video will show you how to create a...,2022-06-03T07:52:29Z,https://i.ytimg.com/vi/ltdJOX_DVtE/maxresdefau...,2022-06-03 07:52:29+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,This short video will show you how to create a...,21,1,None,0,0,0:01:40,100,https://i.ytimg.com/vi/ltdJOX_DVtE/hqdefault.jpg,2022-06-03 07:52:29+00:00
1,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,TIQiMTpzH-Q,⚡️🌎 Naas Community Call (2022/03/03),Here is what we are going to talk about:\n\n⚡️...,2022-03-07T21:00:00Z,https://i.ytimg.com/vi/TIQiMTpzH-Q/maxresdefau...,2022-03-07 21:00:00+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,Here is what we are going to talk about:\n\n⚡️...,54,4,None,0,0,0:51:24,3084,https://i.ytimg.com/vi/TIQiMTpzH-Q/hqdefault.jpg,2022-03-07 21:00:00+00:00
2,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,1kjxWtgeV3E,💬 Using NLP to extract and summarize transcrip...,🌍💬 Interested in Open Source + NLP but don't k...,2021-10-15T18:17:04Z,https://i.ytimg.com/vi/1kjxWtgeV3E/maxresdefau...,2021-10-15 18:17:04+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,🌍💬 Interested in Open Source + NLP but don't k...,549,13,None,0,0,0:07:02,422,https://i.ytimg.com/vi/1kjxWtgeV3E/hqdefault.jpg,2021-10-15 18:17:04+00:00
3,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,lvZKZx9ngrc,Code is power - A brief history of coding.,#code #opensource #power #office #hours #naas,2021-10-03T18:07:15Z,https://i.ytimg.com/vi/lvZKZx9ngrc/maxresdefau...,2021-10-03 18:07:15+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,#code #opensource #power #office #hours #naas,92,10,None,0,2,1:02:56,3776,https://i.ytimg.com/vi/lvZKZx9ngrc/hqdefault.jpg,2021-10-03 18:07:15+00:00
4,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,4ds2FDI_60g,Push Jupyter notebooks to Production ⚡️ with 5...,#naas #notebooks #lowcode #opensource\n\n✊⚡️Pu...,2021-09-16T16:33:00Z,https://i.ytimg.com/vi/4ds2FDI_60g/maxresdefau...,2021-09-16 16:33:00+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,#naas #notebooks #lowcode #opensource\n\n✊⚡️Pu...,1455,33,None,0,7,0:10:47,647,https://i.ytimg.com/vi/4ds2FDI_60g/hqdefault.jpg,2021-09-16 16:33:00+00:00
5,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,tQd-4AsHvqU,Is an idea environmentally friendly? 🌎👍 👎 (Hug...,#huggingface #transformers #ai #model #jupyter...,2021-09-06T17:00:44Z,,2021-09-06 17:00:44+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,#huggingface #transformers #ai #model #jupyter...,38,1,None,0,0,0:15:59,959,https://i.ytimg.com/vi/tQd-4AsHvqU/hqdefault.jpg,2021-09-06 17:00:44+00:00
6,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,jmC5VDN-MOI,🚨 Open Source Contribution step by step (Githu...,#opensource #contribution #git #naas #notebook...,2021-09-01T19:01:49Z,https://i.ytimg.com/vi/jmC5VDN-MOI/maxresdefau...,2021-09-01 19:01:49+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,#opensource #contribution #git #naas #notebook...,346,20,None,0,0,0:43:37,2617,https://i.ytimg.com/vi/jmC5VDN-MOI/hqdefault.jpg,2021-09-01 19:01:49+00:00
7,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,1CfW0N8x8og,🧨 👿 What is wrong with Data Project? ...and wh...,#data #dataops #devops #agile #businesteam #te...,2021-08-17T15:09:55Z,https://i.ytimg.com/vi/1CfW0N8x8og/maxresdefau...,2021-08-17 15:09:55+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,#data #dataops #devops #agile #businesteam #te...,40,2,None,0,0,0:53:54,3234,https://i.ytimg.com/vi/1CfW0N8x8og/hqdefault.jpg,2021-08-17 15:09:55+00:00
8,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,W02XVb8IbGA,⏱1-minute pitch video for Pioneer,A 1-minute video about Naas project.,2021-08-09T16:41:49Z,https://i.ytimg.com/vi/W02XVb8IbGA/maxresdefau...,2021-08-09 16:41:49+00:

### Compute reach

In [7]:
df_uploads = df_uploads.astype({
    'COMMENTCOUNT': int, 
    'LIKECOUNT': int,
    'VIEWCOUNT': int
})
df_uploads['REACH'] = ((df_uploads['COMMENTCOUNT'] + df_uploads['LIKECOUNT'])) / df_uploads['VIEWCOUNT']
df_uploads = df_uploads.round({'REACH': 4})
df_uploads

,CHANNEL_ID_x,PLAYLIST_ID,VIDEO_ID,VIDEO_TITLE,VIDEO_DESCRIPTION,VIDEO_PUBLISHEDAT,VIDEO_THUMBNAILS,PUBLISHEDAT_x,CHANNEL_ID_y,CHANNEL_TITLE,...,VIEWCOUNT,LIKECOUNT,DISLIKECOUNT,FAVORITECOUNT,COMMENTCOUNT,DURATION,DURATION_SECONDS,THUMBNAILS,PUBLISHEDAT_y,REACH
0,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,ltdJOX_DVtE,Create a YouTube API Key (In less than 2 minutes),This short video will show you how to create a...,2022-06-03T07:52:29Z,https://i.ytimg.com/vi/ltdJOX_DVtE/maxresdefau...,2022-06-03 07:52:29+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,21,1,None,0,0,0:01:40,100,https://i.ytimg.com/vi/ltdJOX_DVtE/hqdefault.jpg,2022-06-03 07:52:29+00:00,0.0476
1,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,TIQiMTpzH-Q,⚡️🌎 Naas Community Call (2022/03/03),Here is what we are going to talk about:\n\n⚡️...,2022-03-07T21:00:00Z,https://i.ytimg.com/vi/TIQiMTpzH-Q/maxresdefau...,2022-03-07 21:00:00+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,54,4,None,0,0,0:51:24,3084,https://i.ytimg.com/vi/TIQiMTpzH-Q/hqdefault.jpg,2022-03-07 21:00:00+00:00,0.0741
2,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,1kjxWtgeV3E,💬 Using NLP to extract and summarize transcrip...,🌍💬 Interested in Open Source + NLP but don't k...,2021-10-15T18:17:04Z,https://i.ytimg.com/vi/1kjxWtgeV3E/maxresdefau...,2021-10-15 18:17:04+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,549,13,None,0,0,0:07:02,422,https://i.ytimg.com/vi/1kjxWtgeV3E/hqdefault.jpg,2021-10-15 18:17:04+00:00,0.0237
3,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,lvZKZx9ngrc,Code is power - A brief history of coding.,#code #opensource #power #office #hours #naas,2021-10-03T18:07:15Z,https://i.ytimg.com/vi/lvZKZx9ngrc/maxresdefau...,2021-10-03 18:07:15+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,92,10,None,0,2,1:02:56,3776,https://i.ytimg.com/vi/lvZKZx9ngrc/hqdefault.jpg,2021-10-03 18:07:15+00:00,0.1304
4,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,4ds2FDI_60g,Push Jupyter notebooks to Production ⚡️ with 5...,#naas #notebooks #lowcode #opensource\n\n✊⚡️Pu...,2021-09-16T16:33:00Z,https://i.ytimg.com/vi/4ds2FDI_60g/maxresdefau...,2021-09-16 16:33:00+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,1455,33,None,0,7,0:10:47,647,https://i.ytimg.com/vi/4ds2FDI_60g/hqdefault.jpg,2021-09-16 16:33:00+00:00,0.0275
5,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,tQd-4AsHvqU,Is an idea environmentally friendly? 🌎👍 👎 (Hug...,#huggingface #transformers #ai #model #jupyter...,2021-09-06T17:00:44Z,,2021-09-06 17:00:44+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,38,1,None,0,0,0:15:59,959,https://i.ytimg.com/vi/tQd-4AsHvqU/hqdefault.jpg,2021-09-06 17:00:44+00:00,0.0263
6,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,jmC5VDN-MOI,🚨 Open Source Contribution step by step (Githu...,#opensource #contribution #git #naas #notebook...,2021-09-01T19:01:49Z,https://i.ytimg.com/vi/jmC5VDN-MOI/maxresdefau...,2021-09-01 19:01:49+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,346,20,None,0,0,0:43:37,2617,https://i.ytimg.com/vi/jmC5VDN-MOI/hqdefault.jpg,2021-09-01 19:01:49+00:00,0.0578
7,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,1CfW0N8x8og,🧨 👿 What is wrong with Data Project? ...and wh...,#data #dataops #devops #agile #businesteam #te...,2021-08-17T15:09:55Z,https://i.ytimg.com/vi/1CfW0N8x8og/maxresdefau...,2021-08-17 15:09:55+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,40,2,None,0,0,0:53:54,3234,https://i.ytimg.com/vi/1CfW0N8x8og/hqdefault.jpg,2021-08-17 15:09:55+00:00,0.0500
8,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,W02XVb8IbGA,⏱1-minute pitch video for Pioneer,A 1-minute video about Naas project.,2021-08-09T16:41:49Z,https://i.ytimg.com/vi/W02XVb8IbGA/maxresdefau...,2021-08-09 16:41:49+00:00,UCKKG5hzjXXU_rRdHHWQ8JHQ,naas,...,108,10,None,0,1,0:01:01,61,https://i.ytimg.com/vi/W02XVb8IbGA/hqdefault.jpg,2021-08-09 16:41:49+00:00,0.1019
9,UCKKG5hzjXXU_rRdHHWQ8JHQ,UUKKG5hzjXXU_rRdHHWQ8JHQ,NjUhb92i9eM,🌎🔗Making sense of LinkedIn data: how Mark used...,#socialmedia #linkedin #datamining #datascienc...,2021-07-27T18:07:10Z,https://i.ytimg

### Connect Notion driver

In [8]:
notion.connect(NOTION_TOKEN)

### Get notion database

In [9]:
db = notion.database.query(notion_database_url, query={})

### Helper function to quickly get page from database

In [10]:
def get_page_with_matching_property(db, property_name, property_value):
    return pydash.find(db, lambda x: str(x.properties[property_name]) == property_value)

### Helper function to extract emojis

In [11]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r"\X", text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI["en"] for char in word):
            emoji_list.append(word)
    return emoji_list

def get_tags(text):
        tags = []
        tags_list = re.findall("#[^#| ]+[a-zA-Z0-9]", text)
        for i in range(0, len(tags_list)):
            tag = tags_list[i]
            check_tag = True
            for t in tag:
                if not t.isalpha() and not t.isnumeric() and t != "#":
                    check_tag = False
                if check_tag is False:
                    break
            if check_tag is False:
                tag = tag.rsplit(t)[0]
            tags.append(tag)
        return tags

### Iterate over videos and upsert page in Notion database

In [12]:
page_updated = 0
page_created = 0
page_created_list = []

for _, video in df_uploads.iterrows():
    content_url = f"https://www.youtube.com/watch?v={video['VIDEO_ID']}"
    notion_page = get_page_with_matching_property(db, 'Content URL', content_url)
    if notion_page is None:
        notion_page = notion.page.create(database_id=notion_database_url, title=video['VIDEO_TITLE'])
        page_created += 1
        page_created_list.append(notion_page.url)
        print(f'✅ New notion page create for video: {content_url}')
    else:
        page_updated += 1
        print(f'⚙️ Updating page {notion_page.url} for video {content_url}')
    
    notion_page.link('Content URL', content_url)
    notion_page.number('Engagment score', video['REACH']) # Typo here but it was already there in the database.
    notion_page.number('Engagement score', video['REACH'])
    notion_page.number('Views', video['VIEWCOUNT'])
    notion_page.number('Likes', video['LIKECOUNT'])
    notion_page.number('Comments', video['COMMENTCOUNT'])
    notion_page.date("Publication Date", video['VIDEO_PUBLISHEDAT'])
    
    emojis_array = get_emojis(video['VIDEO_TITLE'] + video['VIDEO_DESCRIPTION'])
    notion_page.rich_text("Emojis", ' ,'.join(emojis_array))
    notion_page.number("Nb emojis", len(emojis_array))
    
    tags_array = get_tags(video['VIDEO_TITLE'] + video['VIDEO_DESCRIPTION'])
    notion_page.rich_text("Tags", ' '.join(tags_array))
    notion_page.number("Nb tags", len(tags_array))
    
    notion_page.select("Status", "Published ✨")
    notion_page.select("Platform", "YouTube")
    notion_page.select("Content type", "Video")
    
    notion_page.update()
    print(f'✅ Page for video {content_url} updated!')

✅ New notion page create for video: https://www.youtube.com/watch?v=ltdJOX_DVtE
✅ Page for video https://www.youtube.com/watch?v=ltdJOX_DVtE updated!
✅ New notion page create for video: https://www.youtube.com/watch?v=TIQiMTpzH-Q
✅ Page for video https://www.youtube.com/watch?v=TIQiMTpzH-Q updated!
✅ New notion page create for video: https://www.youtube.com/watch?v=1kjxWtgeV3E
✅ Page for video https://www.youtube.com/watch?v=1kjxWtgeV3E updated!
✅ New notion page create for video: https://www.youtube.com/watch?v=lvZKZx9ngrc
✅ Page for video https://www.youtube.com/watch?v=lvZKZx9ngrc updated!
✅ New notion page create for video: https://www.youtube.com/watch?v=4ds2FDI_60g
✅ Page for video https://www.youtube.com/watch?v=4ds2FDI_60g updated!
✅ New notion page create for video: https://www.youtube.com/watch?v=tQd-4AsHvqU
✅ Page for video https://www.youtube.com/watch?v=tQd-4AsHvqU updated!
✅ New notion page create for video: https://www.youtube.com/watch?v=jmC5VDN-MOI
✅ Page for video htt

## Output

### Display results

In [13]:
page_created_template = "\n\n".join(page_created_list)

print(f'''
✅ Execution completed!

Number of page created: {page_created}
Number of page updated: {page_updated}

Page created:

{page_created_template}

''')


✅ Execution completed!

Number of page created: 28
Number of page updated: 0

Page created:

https://www.notion.so/Create-a-YouTube-API-Key-In-less-than-2-minutes-16b163e3466c4df0ab00f3ba37aeeebe

https://www.notion.so/Naas-Community-Call-2022-03-03-9e1a2c9f14064168b35cb02fab3f993a

https://www.notion.so/Using-NLP-to-extract-and-summarize-transcript-from-Youtube-e741cdcb91f74b05a3f285bacdd590fd

https://www.notion.so/Code-is-power-A-brief-history-of-coding-48c007cf63414869b6fe1782614a202c

https://www.notion.so/Push-Jupyter-notebooks-to-Production-with-5-low-code-formulas-0a65310207244ce3a5ebc826f84e5503

https://www.notion.so/Is-an-idea-environmentally-friendly-Hugging-Face-API-Naas-4e3cda955a424f738a74ccd476b1868d

https://www.notion.so/Open-Source-Contribution-step-by-step-Github-Naas-76ed5edd854c47afad070b7b81064526

https://www.notion.so/What-is-wrong-with-Data-Project-and-why-DataOps-is-the-key-to-success-dafbddaf56ee403994c5fab91259effb

https://www.notion.so/1-minute-pitch-vid